# Introduction 

This example requires QCoDeS

In [1]:
from collections import defaultdict
import numpy as np 

from visa_mock.base.base_mocker import BaseMocker, scpi
from visa_mock.base.register import register_resource
from qcodes import VisaInstrument, InstrumentChannel
from qcodes.instrument_drivers.stahl import Stahl

In [2]:
class Mocker(BaseMocker):
    """
    A mocker class mocking a multi channel voltage source.
    Voltages are zero by default
    """

    def __init__(self):
        self._voltage = defaultdict(lambda: 0)

    @scpi("\*IDN\?")
    def idn(self): 
        """
        'vendor', 'model', 'serial', 'firmware'
        """
        return "Mocker,testing,00000,0.01"

    # Lets define handler functions. Notice how we can be 
    # lazy in our regular expressions (using ".*"). The 
    # typehints will be used to cast strings to the 
    # required types
    
    @scpi(r":INSTR:CHANNEL(.*):VOLT (.*)")
    def _set_voltage(self, channel: int, value: float) -> None:
        self._voltage[channel] = value

    @scpi(r":INSTR:CHANNEL(.*):VOLT\?")
    def _get_voltage(self, channel: int) -> float:
        return self._voltage[channel]

In [3]:
class MyInstrumentChannel(InstrumentChannel): 
    def __init__(self, parent, name, channel): 
        super().__init__(parent, name)
        
        self.add_parameter(
            "voltage", 
            get_cmd=f":INSTR:CHANNEL{channel}:VOLT?", 
            set_cmd=f":INSTR:CHANNEL{channel}:VOLT {{}}", 
            get_parser=float, 
            unit="V"
        )
        
class MyInstrument(VisaInstrument): 
    
    n_channels = 10
    
    def __init__(self, name, address, **kwargs): 
        super().__init__(name, address, **kwargs)
        
        for channel in range(self.n_channels): 
            
            submodule = MyInstrumentChannel(
                self, 
                f"channel{channel}", 
                channel
            )
            
            self.add_submodule(
                f"channel{channel}", 
                submodule
            )
            
        self.connect_message()

In [4]:
register_resource("MOCK0::mock1::INSTR", Mocker())

In [5]:
my_instrument = MyInstrument("myinst", "MOCK0::mock1::INSTR", visalib="@mock")

Connected to: Mocker testing (serial:00000, firmware:0.01) in 0.01s


In [6]:
my_instrument.channel1.voltage()

0.0

In [7]:
my_instrument.channel1.voltage(2.1)

In [8]:
my_instrument.channel1.voltage()

2.1

In [9]:
my_instrument.channel2.voltage(-2.1)

In [10]:
my_instrument.channel2.voltage()

-2.1

In [11]:
my_instrument.channel1.voltage()

2.1

In [12]:
class BadMocker(BaseMocker):
    """
    A mocker class mocking a multi channel voltage source.
    Voltages are zero by default. 
    
    Note: This code will raise a ValueError
    """

    def __init__(self):
        self._voltage = defaultdict(lambda: 0)

    @scpi("\*IDN\?")
    def idn(self): 
        """
        'vendor', 'model', 'serial', 'firmware'
        """
        return "Mocker,testing,00000,0.01"

    # We show that all handler functions *have* to have
    # annotated arguments.... otherwise a ValueError is 
    # generated. 
    
    @scpi(r":INSTR:CHANNEL(.*):VOLT (.*)")
    def _set_voltage(self, channel: int, value) -> None:
        self._voltage[channel] = value

ValueError: This decorator requires all arguments to be annotated